<a href="https://colab.research.google.com/github/vdadinda/TomTom-PHF-Post-Processing/blob/main/TomTom_PHF_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TomTom Data Post-Processing

This notebook processes TomTom Origin-Destination (O-D) data to generate Peak Hour Factor (PHF) matrices for AM and PM peak periods.

### How to Use

1.  **Input Files and Adjust File Names:** Ensure you have the following files uploaded to your Colab environment:
    *   The TomTom O-D output file in Excel format (`.xlsx`). Update the `file_path` variable in the "Add User Input" section to point to this file.
    *   The TomTom zones definition file in CSV format (`;` delimited). Update the `zones_df` variable in the "Add User Input" section to point to this file.

2.  **Adjust Peak Hour Time Windows:** Define the AM and PM peak hour time windows by modifying the `am_peak_time_window` and `pm_peak_time_window` variables in the "Add User Input" section. The format should match the column headers in your input Excel file (e.g., '08:00 - 09:00').

3.  **Run All Cells:** Execute all the code cells in the notebook sequentially. You can do this by going to "Runtime" -> "Run all" in the Colab menu.

4.  **Output Files:** The notebook will generate two Excel files in your Colab environment:
    *   `PHF_AM.xlsx`: Contains the AM Peak Hour Factor matrix.
    *   `PHF_PM.xlsx`: Contains the PM Peak Hour Factor matrix.
    You can download these files from the "Files" tab on the left-hand side panel.

### Notes

*   Adjust input files in the "Add User Input" section. You can hide all the other sections
*   The code assumes the input Excel file has columns for 'Origin', 'Destination', and trip counts for various time ranges formatted as "Date range: YYYY-MM-DD - YYYY-MM-DD Time range: HH:MM - HH:MM Trips".
*   The `TT_Zones.csv` file is expected to have columns named 'zone', 'type', 'function', and 'category', with the 'function' column indicating 'ori' for origin-only zones and 'des' for destination-only zones.
*   The data cleaning step zeros out trips where the origin is a 'des' zone or the destination is an 'ori' zone, based on the `zones_df`.
*   The PHF matrices are calculated by dividing the peak hour trip counts by the total daily trip counts for each O-D pair.
*   The code includes checks to ensure the sums of trips in the adjusted DataFrame match the sums in the generated peak hour matrices.

### Import Packages

In [ ]:
#Import packages

import pandas as pd
import numpy as np
from pandas import pivot_table

### **Add User Input**

In [ ]:
# Define the time window for the AM and PM Peaks
am_peak_time_window = '08:00 - 09:00'
pm_peak_time_window = '16:00 - 17:00'

# Add input filepath
file_path = "/content/Nedlands Test OD_V4 (Per, JC) - Exclude Intra & Via.xlsx" #TomTom O-D output
zones_df = pd.read_csv("/content/TT_Zones.csv", sep=';', dtype={'zone': 'str'}) #TomTom zone origin/destination only definition

### Read and Display User Input

In [32]:
# Read and display first 5 rows
input_df = pd.read_excel(file_path, dtype={'Origin': 'str', 'Destination': 'str'})
display(input_df.head())

zones_df = zones_df[['zone', 'type', 'function', 'category']]
display(zones_df.head())

,Origin,Destination,Date range: 2024-04-01 - 2025-03-31 Time range: 00:00 - 01:00 Trips,Date range: 2024-04-01 - 2025-03-31 Time range: 00:00 - 01:00 Percent,Date range: 2024-04-01 - 2025-03-31 Time range: 01:00 - 02:00 Trips,Date range: 2024-04-01 - 2025-03-31 Time range: 01:00 - 02:00 Percent,Date range: 2024-04-01 - 2025-03-31 Time range: 02:00 - 03:00 Trips,Date range: 2024-04-01 - 2025-03-31 Time range: 02:00 - 03:00 Percent,Date range: 2024-04-01 - 2025-03-31 Time range: 03:00 - 04:00 Trips,Date range: 2024-04-01 - 2025-03-31 Time range: 03:00 - 04:00 Percent,...,Date range: 2025-03-27 - 2025-03-27 Time range: 19:00 - 20:00 Trips,Date range: 2025-03-27 - 2025-03-27 Time range: 19:00 - 20:00 Percent,Date range: 2025-03-27 - 2025-03-27 Time range: 20:00 - 21:00 Trips,Date range: 2025-03-27 - 2025-03-27 Time range: 20:00 - 21:00 Percent,Date range: 2025-03-27 - 2025-03-27 Time range: 21:00 - 22:00 Trips,Date range: 2025-03-27 - 2025-03-27 Time range: 21:00 - 22:00 Percent,Date range: 2025-03-27 - 2025-03-27 Time range: 22:00 - 23:00 Trips,Date range: 2025-03-27 - 2025-03-27 Time range: 22:00 - 23:00 Percent,Date range: 2025-03-27 - 2025-03-27 Time range: 23:00 - 00:00 Trips,Date range: 2025-03-27 - 2025-03-27 Time range: 23:00 - 00:00 Percent
0,022.03,022.03,0,0.000,0,0.0,0,0.0,0,0.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
1,022.03,016.9,0,0.000,0,0.0,0,0.0,0,0.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
2,022.03,018.3,3,0.009,0,0.0,0,0.0,0,0.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
3,022.03,021.02,1,0.003,0,0.0,0,0.0,0,0.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
4,022.03,021.20,0,0.000,0,0.0,0,0.0,0,0.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0


,zone,type,function,category
0,2001,ext,des,maj
1,2002,ext,ori,maj
2,2003,ext,des,maj
3,2004,ext,ori,maj
4,2005,ext,des,maj


In [33]:
# Identify the trip and percentage columns
numeric_cols = [col for col in input_df.columns if 'Trips' in col or 'Percent' in col]

# Filter the DataFrame and then select only the numeric columns before summing
total_all_sample = input_df[(input_df['Origin'] == '001.1') & (input_df['Destination'] == '001.2')][numeric_cols].sum(axis=1)

# AM Peak - use the full column name
am_peak_col_name = f"Date range: 2024-04-01 - 2025-03-31 Time range: {am_peak_time_window} Trips"
am_sample = input_df[(input_df['Origin'] == '001.1') & (input_df['Destination'] == '001.2')][am_peak_col_name]

display(total_all_sample)
display(am_sample)
print(am_sample/total_all_sample)

,0
21101,456.509


,Date range: 2024-04-01 - 2025-03-31 Time range: 08:00 - 09:00 Trips
21101,23


21101    0.050382
dtype: float64


### Data Cleaning: rename fields, clean unwanted columns

In [34]:
# Select 'Origin' and 'Destination' columns
selected_columns = ['Origin', 'Destination']

# Identify columns that match the pattern "Date range: 2024-04-01 - 2025-03-31 Time range: <an hour window> Trips"
trips_columns = [col for col in input_df.columns if 'Date range: 2024-04-01 - 2025-03-31 Time range:' in col and 'Trips' in col]

# Combine the selected columns
selected_columns.extend(trips_columns)

# Create a new DataFrame with only the selected columns
filtered_df = input_df[selected_columns].copy() # Add .copy() to avoid SettingWithCopyWarning

# Rename the trips columns to only show the hour window
new_column_names = {}
for col in trips_columns:
    # Extract the time range using string manipulation
    start_index = col.find('Time range:') + len('Time range:') + 1
    end_index = col.find(' Trips')
    time_range = col[start_index:end_index].strip()
    new_column_names[col] = time_range

filtered_df = filtered_df.rename(columns=new_column_names)

# Display the first few rows of the filtered DataFrame
print(f"filtered_df no. of rows, columns: {filtered_df.shape}")
# display(filtered_df.columns)
display(filtered_df.head())

# Identify the trip columns in filtered_df
trip_columns_filtered = [col for col in filtered_df.columns if col not in ['Origin', 'Destination']]

# Calculate the sum of all trip values in filtered_df
total_trips_filtered_df = filtered_df[trip_columns_filtered].sum().sum()

# Display the sum of all trip values in filtered_df
print(f"\nSum of all trip values in filtered_df: {total_trips_filtered_df}")

filtered_df no. of rows, columns: (55696, 26)


,Origin,Destination,00:00 - 01:00,01:00 - 02:00,02:00 - 03:00,03:00 - 04:00,04:00 - 05:00,05:00 - 06:00,06:00 - 07:00,07:00 - 08:00,...,14:00 - 15:00,15:00 - 16:00,16:00 - 17:00,17:00 - 18:00,18:00 - 19:00,19:00 - 20:00,20:00 - 21:00,21:00 - 22:00,22:00 - 23:00,23:00 - 00:00
0,022.03,022.03,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,022.03,016.9,0,0,0,0,0,0,2,0,...,0,3,4,1,0,0,0,0,0,0
2,022.03,018.3,3,0,0,0,1,2,5,13,...,7,17,17,8,10,6,5,6,3,3
3,022.03,021.02,1,0,0,0,0,0,2,2,...,1,3,1,2,2,0,0,3,0,1
4,022.03,021.20,0,0,0,0,0,0,2,3,...,0,3,2,4,0,1,0,1,0,0



Sum of all trip values in filtered_df: 8302611


### Data Cleaning: zero out O/D only zones

Adjust the `filtered_df` DataFrame based on zone functions defined in the "/content/TT_Zones.csv" file. If a zone is designated as 'des' (destination only) in "/content/TT_Zones.csv", zero out all trip values in `filtered_df` where this zone is the origin. If a zone is designated as 'ori' (origin only) in "/content/TT_Zones.csv", zero out all trip values in `filtered_df` where this zone is the destination. Display the first few rows of the adjusted `filtered_df`.

In [35]:
#Identify origin and destination only zones

des_only_zones = zones_df[zones_df['function'] == 'des']['zone'].tolist()
ori_only_zones = zones_df[zones_df['function'] == 'ori']['zone'].tolist()

print("Des Only Zones (first 5):", des_only_zones[:5])
print("Ori Only Zones (first 5):", ori_only_zones[:5])

Des Only Zones (first 5): ['2001', '2003', '2005', '2007', '2009']
Ori Only Zones (first 5): ['2002', '2004', '2006', '2008', '2010']


In [36]:
#Initialize adjusted dataframe

adj_fil_df = filtered_df.copy(deep=True)
display(adj_fil_df.head())

,Origin,Destination,00:00 - 01:00,01:00 - 02:00,02:00 - 03:00,03:00 - 04:00,04:00 - 05:00,05:00 - 06:00,06:00 - 07:00,07:00 - 08:00,...,14:00 - 15:00,15:00 - 16:00,16:00 - 17:00,17:00 - 18:00,18:00 - 19:00,19:00 - 20:00,20:00 - 21:00,21:00 - 22:00,22:00 - 23:00,23:00 - 00:00
0,022.03,022.03,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,022.03,016.9,0,0,0,0,0,0,2,0,...,0,3,4,1,0,0,0,0,0,0
2,022.03,018.3,3,0,0,0,1,2,5,13,...,7,17,17,8,10,6,5,6,3,3
3,022.03,021.02,1,0,0,0,0,0,2,2,...,1,3,1,2,2,0,0,3,0,1
4,022.03,021.20,0,0,0,0,0,0,2,3,...,0,3,2,4,0,1,0,1,0,0


In [37]:
# Identify the trip columns in filtered_df
trip_columns_filtered = [col for col in filtered_df.columns if col not in ['Origin', 'Destination']]

# Convert des_only_zones and ori_only_zones to string type for consistent comparison
des_only_zones_str = [str(zone) for zone in des_only_zones]
ori_only_zones_str = [str(zone) for zone in ori_only_zones]

# Use boolean indexing to identify rows where origin is a des_only zone or destination is an ori_only zone
condition = adj_fil_df['Origin'].isin(des_only_zones_str) | adj_fil_df['Destination'].isin(ori_only_zones_str)

# Set the trip values to zero for the rows that meet the condition
adj_fil_df.loc[condition, trip_columns_filtered] = 0

# Display the first few rows of the adjusted DataFrame
display(adj_fil_df.head())

,Origin,Destination,00:00 - 01:00,01:00 - 02:00,02:00 - 03:00,03:00 - 04:00,04:00 - 05:00,05:00 - 06:00,06:00 - 07:00,07:00 - 08:00,...,14:00 - 15:00,15:00 - 16:00,16:00 - 17:00,17:00 - 18:00,18:00 - 19:00,19:00 - 20:00,20:00 - 21:00,21:00 - 22:00,22:00 - 23:00,23:00 - 00:00
0,022.03,022.03,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,022.03,016.9,0,0,0,0,0,0,2,0,...,0,3,4,1,0,0,0,0,0,0
2,022.03,018.3,3,0,0,0,1,2,5,13,...,7,17,17,8,10,6,5,6,3,3
3,022.03,021.02,1,0,0,0,0,0,2,2,...,1,3,1,2,2,0,0,3,0,1
4,022.03,021.20,0,0,0,0,0,0,2,3,...,0,3,2,4,0,1,0,1,0,0


In [38]:
# Check if query worked

# Before
# Filter samples_before_df based on the specified conditions
samples_before_df = filtered_df[
    ((filtered_df['Origin'] == '2014') & (filtered_df['Destination'] == '002.09')) |
    ((filtered_df['Origin'] == '002.08') & (filtered_df['Destination'] == '2017'))
]

# Display the extracted data
display(samples_before_df)

# After
# Filter adj_fil_df based on the specified conditions
samples_after_df = adj_fil_df[
    ((adj_fil_df['Origin'] == '2014') & (adj_fil_df['Destination'] == '002.09')) |
    ((adj_fil_df['Origin'] == '002.08') & (adj_fil_df['Destination'] == '2017'))
]

# Display the extracted data
display(samples_after_df)

,Origin,Destination,00:00 - 01:00,01:00 - 02:00,02:00 - 03:00,03:00 - 04:00,04:00 - 05:00,05:00 - 06:00,06:00 - 07:00,07:00 - 08:00,...,14:00 - 15:00,15:00 - 16:00,16:00 - 17:00,17:00 - 18:00,18:00 - 19:00,19:00 - 20:00,20:00 - 21:00,21:00 - 22:00,22:00 - 23:00,23:00 - 00:00
5616,002.08,2017,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,1
43446,2014,002.09,0,0,1,0,0,0,0,0,...,0,1,1,0,1,0,1,1,0,0


,Origin,Destination,00:00 - 01:00,01:00 - 02:00,02:00 - 03:00,03:00 - 04:00,04:00 - 05:00,05:00 - 06:00,06:00 - 07:00,07:00 - 08:00,...,14:00 - 15:00,15:00 - 16:00,16:00 - 17:00,17:00 - 18:00,18:00 - 19:00,19:00 - 20:00,20:00 - 21:00,21:00 - 22:00,22:00 - 23:00,23:00 - 00:00
5616,002.08,2017,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43446,2014,002.09,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Matrix Conversion: All Day

In [39]:
# Identify the trip columns in adj_fil_df
trip_columns = [col for col in adj_fil_df.columns if col not in ['Origin', 'Destination']]

# Aggregate trip values by summing across trip columns
# Drop 'Total_Trips' column if it already exists to prevent double counting
if 'Total_Trips' in adj_fil_df.columns:
    adj_fil_df = adj_fil_df.drop(columns=['Total_Trips'])

adj_fil_df['Total_Trips'] = adj_fil_df[trip_columns].sum(axis=1)

# Create the Origin-Destination matrix using pivot_table
od_matrix_all = pivot_table(adj_fil_df, values='Total_Trips', index='Origin', columns='Destination')

# Fill any missing values with 0
od_matrix_all = od_matrix_all.fillna(0)

#Check total trips
total_trips_all = od_matrix_all.sum().sum()

# Display the first few rows of the resulting Origin-Destination matrix
display(od_matrix_all.head())
display(total_trips_all)

Destination,001.1,001.2,001.3,002.01,002.02,002.04,002.05,002.06,002.07,002.08,...,3032,3033,3034,3035,3036,3037,3038,3039,3040,3041
Origin,,,,,,,,,,,,,,,,,,,,,
001.1,0.0,397.0,1133.0,96.0,263.0,129.0,200.0,50.0,162.0,61.0,...,3.0,3.0,16.0,12.0,7.0,338.0,168.0,3.0,0.0,12.0
001.2,446.0,0.0,865.0,134.0,326.0,109.0,143.0,92.0,151.0,36.0,...,2.0,8.0,55.0,45.0,34.0,651.0,629.0,28.0,0.0,28.0
001.3,261.0,801.0,0.0,460.0,1333.0,476.0,522.0,293.0,819.0,126.0,...,1.0,2.0,34.0,24.0,13.0,534.0,685.0,40.0,0.0,89.0
002.01,59.0,85.0,242.0,0.0,307.0,293.0,262.0,422.0,218.0,44.0,...,1.0,4.0,18.0,4.0,4.0,114.0,125.0,3.0,0.0,13.0
002.02,134.0,568.0,375.0,321.0,0.0,361.0,290.0,766.0,3117.0,689.0,...,2.0,9.0,32.0,9.0,12.0,661.0,257.0,7.0,0.0,108.0


np.float64(8275389.0)

### Matrix Conversion: AM Peak

In [40]:
# Select only the specified time window column
am_peak_trips = adj_fil_df[['Origin', 'Destination', am_peak_time_window]].copy()

# Rename the selected column to 'AM Peak'
am_peak_trips = am_peak_trips.rename(columns={am_peak_time_window: 'AM Peak'})

# Create the Origin-Destination matrix for AM Peak using pivot_table
od_matrix_am_peak = pivot_table(am_peak_trips, values='AM Peak', index='Origin', columns='Destination')

# Fill any missing values with 0
od_matrix_am_peak = od_matrix_am_peak.fillna(0)

#Check total trips
total_trips_am_peak = od_matrix_am_peak.sum().sum()

# Display the first few rows of the resulting AM Peak Origin-Destination matrix
display(od_matrix_am_peak.head())

# Display the sum of all trip values in Matrix AM Peak
print(f"\nSum of all trip values in Matrix AM Peak: {od_matrix_am_peak.sum().sum()}")

Destination,001.1,001.2,001.3,002.01,002.02,002.04,002.05,002.06,002.07,002.08,...,3032,3033,3034,3035,3036,3037,3038,3039,3040,3041
Origin,,,,,,,,,,,,,,,,,,,,,
001.1,0.0,23.0,83.0,12.0,19.0,4.0,10.0,1.0,9.0,3.0,...,0.0,0.0,1.0,0.0,0.0,27.0,6.0,0.0,0.0,1.0
001.2,33.0,0.0,68.0,20.0,30.0,2.0,4.0,7.0,6.0,0.0,...,0.0,0.0,1.0,5.0,0.0,41.0,26.0,2.0,0.0,3.0
001.3,11.0,33.0,0.0,79.0,121.0,12.0,27.0,20.0,80.0,4.0,...,0.0,0.0,1.0,1.0,0.0,18.0,45.0,2.0,0.0,6.0
002.01,1.0,1.0,6.0,0.0,12.0,8.0,12.0,20.0,8.0,0.0,...,1.0,0.0,1.0,0.0,0.0,5.0,1.0,0.0,0.0,0.0
002.02,4.0,36.0,27.0,25.0,0.0,8.0,14.0,85.0,249.0,50.0,...,0.0,1.0,1.0,0.0,2.0,11.0,3.0,0.0,0.0,9.0



Sum of all trip values in Matrix AM Peak: 708253.0


### Matrix Conversion: PM Peak

In [41]:
# Select only the specified time window column
pm_peak_trips = adj_fil_df[['Origin', 'Destination', pm_peak_time_window]].copy()

# Rename the selected column to 'PM Peak'
pm_peak_trips = pm_peak_trips.rename(columns={pm_peak_time_window: 'PM Peak'})

# Create the Origin-Destination matrix for PM Peak using pivot_table
od_matrix_pm_peak = pivot_table(pm_peak_trips, values='PM Peak', index='Origin', columns='Destination')

# Fill any missing values with 0
od_matrix_pm_peak = od_matrix_pm_peak.fillna(0)

# Display the first few rows of the resulting PM Peak Origin-Destination matrix
display(od_matrix_pm_peak.head())

# Display the sum of all trip values in Matrix PM Peak
print(f"\nSum of all trip values in Matrix PM Peak: {od_matrix_pm_peak.sum().sum()}")

Destination,001.1,001.2,001.3,002.01,002.02,002.04,002.05,002.06,002.07,002.08,...,3032,3033,3034,3035,3036,3037,3038,3039,3040,3041
Origin,,,,,,,,,,,,,,,,,,,,,
001.1,0.0,27.0,128.0,10.0,18.0,9.0,14.0,8.0,10.0,4.0,...,0.0,1.0,1.0,2.0,0.0,42.0,16.0,0.0,0.0,0.0
001.2,24.0,0.0,94.0,10.0,23.0,12.0,9.0,7.0,15.0,3.0,...,0.0,0.0,5.0,2.0,1.0,64.0,132.0,3.0,0.0,1.0
001.3,16.0,55.0,0.0,40.0,127.0,34.0,43.0,31.0,75.0,13.0,...,0.0,0.0,4.0,2.0,0.0,54.0,95.0,2.0,0.0,7.0
002.01,6.0,6.0,24.0,0.0,29.0,17.0,18.0,26.0,10.0,4.0,...,0.0,0.0,7.0,0.0,0.0,13.0,32.0,0.0,0.0,0.0
002.02,11.0,42.0,30.0,27.0,0.0,22.0,21.0,37.0,160.0,44.0,...,0.0,0.0,5.0,0.0,2.0,79.0,52.0,0.0,0.0,7.0



Sum of all trip values in Matrix PM Peak: 658966.0


### Matrix PHF: AM Peak

In [42]:
# Ensure the matrices have the same shape and indices before division
if not od_matrix_am_peak.shape == od_matrix_all.shape or not od_matrix_am_peak.index.equals(od_matrix_all.index) or not od_matrix_am_peak.columns.equals(od_matrix_all.columns):
    print("Warning: AM Peak and All Day matrices have different shapes or indices. Division might not be accurate.")
    # Attempt to reindex the matrices to align them (optional, depending on desired behavior)
    # od_matrix_am_peak = od_matrix_am_peak.align(od_matrix_all, fill_value=0)[0]
    # od_matrix_all = od_matrix_all.align(od_matrix_am_peak, fill_value=0)[1]

# Create the PHF_AM matrix by dividing AM Peak by All Day
# Use .divide() with fill_value=0 to handle division by zero
PHF_AM = od_matrix_am_peak.divide(od_matrix_all, fill_value=0)

# Replace any remaining NaN values with 0
PHF_AM = PHF_AM.fillna(0)

# Convert the matrix values to floats
PHF_AM = PHF_AM.astype(float)

# Display the first few rows of the resulting PHF_AM matrix
display(PHF_AM.head())

Destination,001.1,001.2,001.3,002.01,002.02,002.04,002.05,002.06,002.07,002.08,...,3032,3033,3034,3035,3036,3037,3038,3039,3040,3041
Origin,,,,,,,,,,,,,,,,,,,,,
001.1,0.000000,0.057935,0.073257,0.125000,0.072243,0.031008,0.050000,0.020000,0.055556,0.049180,...,0.0,0.000000,0.062500,0.000000,0.000000,0.079882,0.035714,0.000000,0.0,0.083333
001.2,0.073991,0.000000,0.078613,0.149254,0.092025,0.018349,0.027972,0.076087,0.039735,0.000000,...,0.0,0.000000,0.018182,0.111111,0.000000,0.062980,0.041335,0.071429,0.0,0.107143
001.3,0.042146,0.041199,0.000000,0.171739,0.090773,0.025210,0.051724,0.068259,0.097680,0.031746,...,0.0,0.000000,0.029412,0.041667,0.000000,0.033708,0.065693,0.050000,0.0,0.067416
002.01,0.016949,0.011765,0.024793,0.000000,0.039088,0.027304,0.045802,0.047393,0.036697,0.000000,...,1.0,0.000000,0.055556,0.000000,0.000000,0.043860,0.008000,0.000000,0.0,0.000000
002.02,0.029851,0.063380,0.072000,0.077882,0.000000,0.022161,0.048276,0.110966,0.079885,0.072569,...,0.0,0.111111,0.031250,0.000000,0.166667,0.016641,0.011673,0.000000,0.0,0.083333


In [43]:
# Select only the specified time window column
pm_peak_trips = adj_fil_df[['Origin', 'Destination', pm_peak_time_window]].copy()

# Rename the selected column to 'PM Peak'
pm_peak_trips = pm_peak_trips.rename(columns={pm_peak_time_window: 'PM Peak'})

# Create the Origin-Destination matrix for PM Peak using pivot_table
od_matrix_pm_peak = pivot_table(pm_peak_trips, values='PM Peak', index='Origin', columns='Destination')

# Fill any missing values with 0
od_matrix_pm_peak = od_matrix_pm_peak.fillna(0)

# Ensure the matrices have the same shape and indices before division
if not od_matrix_pm_peak.shape == od_matrix_all.shape or not od_matrix_pm_peak.index.equals(od_matrix_all.index) or not od_matrix_pm_peak.columns.equals(od_matrix_all.columns):
    print("Warning: PM Peak and All Day matrices have different shapes or indices. Division might not be accurate.")
    # Attempt to reindex the matrices to align them (optional, depending on desired behavior)
    # od_matrix_pm_peak = od_matrix_pm_peak.align(od_matrix_all, fill_value=0)[0]
    # od_matrix_all = od_matrix_all.align(od_matrix_am_peak, fill_value=0)[1]

# Create the PHF_PM matrix by dividing PM Peak by All Day
# Use .divide() with fill_value=0 to handle division by zero
PHF_PM = od_matrix_pm_peak.divide(od_matrix_all, fill_value=0)

# Replace any remaining NaN values with 0
PHF_PM = PHF_PM.fillna(0)

# Convert the matrix values to floats
PHF_PM = PHF_PM.astype(float)

# Display the first few rows of the resulting PHF_AM matrix
display(PHF_PM.head())

Destination,001.1,001.2,001.3,002.01,002.02,002.04,002.05,002.06,002.07,002.08,...,3032,3033,3034,3035,3036,3037,3038,3039,3040,3041
Origin,,,,,,,,,,,,,,,,,,,,,
001.1,0.000000,0.068010,0.112974,0.104167,0.068441,0.069767,0.070000,0.160000,0.061728,0.065574,...,0.0,0.333333,0.062500,0.166667,0.000000,0.124260,0.095238,0.000000,0.0,0.000000
001.2,0.053812,0.000000,0.108671,0.074627,0.070552,0.110092,0.062937,0.076087,0.099338,0.083333,...,0.0,0.000000,0.090909,0.044444,0.029412,0.098310,0.209857,0.107143,0.0,0.035714
001.3,0.061303,0.068664,0.000000,0.086957,0.095274,0.071429,0.082375,0.105802,0.091575,0.103175,...,0.0,0.000000,0.117647,0.083333,0.000000,0.101124,0.138686,0.050000,0.0,0.078652
002.01,0.101695,0.070588,0.099174,0.000000,0.094463,0.058020,0.068702,0.061611,0.045872,0.090909,...,0.0,0.000000,0.388889,0.000000,0.000000,0.114035,0.256000,0.000000,0.0,0.000000
002.02,0.082090,0.073944,0.080000,0.084112,0.000000,0.060942,0.072414,0.048303,0.051331,0.063861,...,0.0,0.000000,0.156250,0.000000,0.166667,0.119516,0.202335,0.000000,0.0,0.064815


### Generate Output CSV Files

In [44]:
# Export PHF_AM matrix to an Excel file
PHF_AM.to_excel('PHF_AM.xlsx')

# Export PHF_PM matrix to an Excel file
PHF_PM.to_excel('PHF_PM.xlsx')

print("PHF_AM.xlsx and PHF_PM.xlsx have been saved.")

PHF_AM.xlsx and PHF_PM.xlsx have been saved.


### Checks

In [ ]:
# Calculate the sum of the '08:00 - 09:00' column in adj_fil_df
sum_am_peak_adj_df = adj_fil_df[am_peak_time_window].sum()

# Calculate the sum of all values in the od_matrix_am_peak
sum_am_peak_matrix = od_matrix_am_peak.sum().sum()

# Display the sums for comparison
print(f"Sum of '{am_peak_time_window}' column in adj_fil_df: {sum_am_peak_adj_df}")
print(f"Sum of all values in od_matrix_am_peak: {sum_am_peak_matrix}")

if abs(sum_am_peak_adj_df - sum_am_peak_matrix) < 1e-9: # Use a small tolerance for float comparison
    print("\nThe sums match.")
else:
    print("\nThe sums do not match. There might be an issue in the matrix creation.")

In [46]:
# Filter the DataFrame and then select only the numeric columns before summing
total_all_sample = adj_fil_df[(adj_fil_df['Origin'] == '001.1') & (adj_fil_df['Destination'] == '001.2')]['Total_Trips']

# AM Peak - use the full column name
am_peak_col_name = am_peak_time_window
am_sample = adj_fil_df[(adj_fil_df['Origin'] == '001.1') & (adj_fil_df['Destination'] == '001.2')][am_peak_col_name]

display(total_all_sample)
display(am_sample)
print(am_sample/total_all_sample)

,Total_Trips
21101,397


,08:00 - 09:00
21101,23


21101    0.057935
dtype: float64
